
# Import Data

In [ ]:
# Program extracting first column 
import xlrd

loc = ("./utils/Modified_Bioinfer_label.xlsx")

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0) 
sheet.cell_value(0, 0) 
PPI = []

for i in range(1, sheet.nrows): 
    PPI.append([sheet.cell_value(i, 4), sheet.cell_value(i, 5), sheet.cell_value(i, 6)])

In [ ]:
# unique_PPI = [list(x) for x in set(tuple(x) for x in PPI)]

In [ ]:
unique_PPI = PPI

# Extract PDB Files from Genes

In [ ]:
from pypdb import *
import Bio
from Bio.PDB import PDBList
pdbl = PDBList()

In [ ]:
protein_gene = {}
def gene_to_pdb(pdb_id):
    found_pdbs = Query(pdb_id).search()
    print (found_pdbs)
    temp= []
    for j in found_pdbs:
        temp.append(int(describe_pdb(j)['nr_residues']))
    final_pdb_id = found_pdbs[temp.index(max(temp))]
    protein_gene[pdb_id] = final_pdb_id
    print (final_pdb_id)
    pdbl.retrieve_pdb_file(final_pdb_id,pdir='PDB', file_format='pdb')

In [ ]:
unique_protein = []
for proteins in unique_PPI:
    unique_protein.append(proteins[0])
    unique_protein.append(proteins[1])

In [ ]:
# unique_protein = list(set(unique_protein))
# print(len(unique_protein), "instances of unique proteins found from", len(unique_PPI), "documents")

In [ ]:
len(unique_PPI) // 25

In [ ]:
# for ppi in unique_protein:
#     gene_to_pdb(ppi)
#     if len(protein_gene) % 50 == 0:
#         with open('protein_gene.txt', 'w') as f:
#             f.write(str(protein_gene))

# Node formatting

In [ ]:
import ast

with open('./utils/protein_pdb.txt', 'r') as file:
    contents = file.read()
    protein_pdb = ast.literal_eval(contents)

In [ ]:
import numpy as np
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(["N", "C", "O", "H", "S", "N1+"])
    
def get_ms_feature_vector(gene):
    if(gene is None):
        return([])
    temp = []
    try:
        with open('./PDB/' + gene +'.pdb', 'r') as f1:
            lines = f1.readlines()
            for line in lines:
                split_ = line.split()
                if split_[0] == 'ATOM':
                    temp.extend([float(split_[6]), float(split_[7]), float(split_[8])])  
                    temp.extend(label_encoder.transform([split_[-1]]))#.toarray().squeeze().tolist())
    except:
        pass
    return(temp)

In [ ]:
from tqdm import tqdm

for x in tqdm(range(0, 1)):
    nodes = []
    labels = []
    feature_vector = {}
    for i in range(x * len(unique_PPI) // 25, (x + 1) * len(unique_PPI) // 25):
        proteins = unique_PPI[i]
        temp = []
        try:
            gene_1 = protein_pdb[proteins[0]]
        except:
            gene_1 = None
        try:
            gene_2 = protein_pdb[proteins[1]]
        except:
            gene_2 = None
        if proteins[0] not in feature_vector:
            feature_vector[gene_1] = get_ms_feature_vector(gene_1)
        temp.extend(feature_vector[gene_1])
        temp.extend([0] * (431520 - len(temp)))
        if proteins[1] not in feature_vector:
            feature_vector[gene_2] = get_ms_feature_vector(gene_2)
        temp.extend(feature_vector[gene_2])
        temp.extend([0] * (863040 - len(temp)))
        nodes.append(temp)
        labels.append(float(proteins[2]))
    encoding = []
    nodes_tens = torch.tensor(nodes)
    nodes_tens = pre_process(nodes_tens)
    for i in tqdm(range(0, len(nodes_tens), 5)):
        encoding.extend(encoder.forward(nodes_tens[i:i + 5]).tolist())
    del nodes_tens
    with open("./utils/node_enc{}.csv".format(x), "w") as f:
        writer = csv.writer(f)
        writer.writerows(encoding)

In [ ]:
min([len(node) for node in nodes])

In [ ]:
print(len(nodes))
print(len(labels))

In [ ]:
# with open('./utils/unique_ppi.txt', 'w') as filehandle:
#     for listitem in unique_PPI:
#         filehandle.write('%s\n' % listitem)

In [ ]:
# import csv

# with open("./utils/node.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(nodes)

# with open("./utils/label.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerow(labels)

## CNN Encoder

In [ ]:
import csv

In [ ]:
# nodes = []
encoding = []
# labels = []

In [ ]:
# with open('./utils/node.csv', 'r') as csv_file:
#     csv_reader = csv.reader(csv_file)
#     line_count = 0
#     for row in csv_reader:
#         nodes.extend([float(val) for val in row])

In [ ]:
# with open('./utils/label.csv', 'r') as csv_file:
#     csv_reader = csv.reader(csv_file)
#     line_count = 0
#     for row in csv_reader:
#         labels.extend([float(val) for val in row])

In [ ]:
print(len(nodes))
print(len(labels))

In [ ]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout

In [ ]:
def pre_process(tens):
    tens = torch.reshape(tens, (tens.shape[0], -1, 4))
    tens = torch.unsqueeze(tens, 0)
    tens = torch.transpose(tens, 0, 1)
    return(tens)

In [ ]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
#             MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
#             MaxPool2d(kernel_size=2, stride=2),
            Conv2d(4, 4, kernel_size=5, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(431516, 1185)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
encoder = Net()

In [ ]:
print(encoder)

In [ ]:
nodes_tens = torch.tensor(nodes)
nodes_tens = pre_process(nodes_tens)

In [ ]:
for i in tqdm(range(0, len(nodes_tens), 5)):
    encoding.extend(encoder.forward(nodes_tens[i:i + 5]).tolist())

In [ ]:
del nodes_tens

In [ ]:
print(len(labels))
print(len(encoding))

In [ ]:
with open("./utils/node_enc{}.csv".format(a), "w") as f:
    writer = csv.writer(f)
    writer.writerows(encoding)
a=a+1

## Loading from CSV to TXT

In [ ]:
import csv
nodes_enc = []

In [ ]:
for x in range(0, 25):        
    with open("./utils/node_enc{}.csv".format(x), "r") as csv_file:
        csv_reader = csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if(len(row) != 0):
                nodes_enc.append([float(val) for val in row])

In [ ]:
len(nodes_enc)

In [ ]:
lines = []

for node in nodes_enc:
    temp = "\t".join([str(elem) for elem in node])
    lines.append(temp + '\n')

In [ ]:
with open('./utils/node_MS', 'w') as f:
    f.writelines(lines)

# Edge Formatting

In [ ]:
# define empty list
unique_PPI = []

# open file and read the content in a list
with open('./utils/unique_ppi.txt', 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        temp = []
        # remove linebreak which is the last character of the string
        line = line[:-1]
        line = line.strip('[').strip(']').split(", ")
        temp = [item.replace("'", "") for item in line]
        # add item to the list
        unique_PPI.append(temp)

In [ ]:
len(unique_PPI)

In [ ]:
import ast

with open('./utils/protein_gene.txt', 'r') as file:
    contents = file.read()
    protein_gene = ast.literal_eval(contents)

In [ ]:
edges = []
for i in tqdm_notebook(range(0, len(unique_PPI))):
    for j in range(i + 1, len(unique_PPI)):
        expt_1 = unique_PPI[i]
        expt_2 = unique_PPI[j]
        genes_1 = [protein_gene[unique_PPI[i][0]], protein_gene[unique_PPI[i][1]]]
        genes_2 = [protein_gene[unique_PPI[j][0]], protein_gene[unique_PPI[j][1]]]
        if(len([x for x in genes_1 if x in genes_2])):
            edges.append(str(i) + "\t" + str(j) + "\n")

In [ ]:
with open('./utils/link_MS', 'w') as f:
    f.writelines(edges)

# Batching

In [ ]:
# with open('./utils/node_text', 'r') as f:
#     lines_node_text = f.readlines()
    
# f.close()

# with open('./utils/node_MS', 'r') as f:
#     lines_node_ms = f.readlines()
    
# f.close()

# lines = lines_node_ms + lines_node_text
    
# with open('node', 'w') as f:
#     f.writelines(lines)

# f.close()

In [ ]:
# with open('./utils/link_text', 'r') as f:
#     lines_link_text = f.readlines()
    
# f.close()

# with open('./utils/link_MS', 'r') as f:
#     lines_link_ms = f.readlines()
    
# f.close()

# with open('link', 'w') as f:
#     f.writelines(lines_link_ms)
#     f.writelines(lines_link_text)

# f.close()